In [ ]:
!pip install torch torchvision scikit-learn pandas matplotlib kaggle imbalanced-learn

In [ ]:
import sys
!{sys.executable} -m pip install imbalanced-learn


In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Đọc dữ liệu từ file CSV
data = pd.read_csv('creditcard.csv')

# Chuẩn hóa cột 'Amount' và 'Time'
scaler = StandardScaler()
data['Amount_scaled'] = scaler.fit_transform(data[['Amount']])
data['Time_scaled'] = scaler.fit_transform(data[['Time']])

# Xóa các cột 'Class', 'Amount', và 'Time' chưa chuẩn hóa
X = data.drop(columns=['Class', 'Amount', 'Time'])
X['Amount'] = data['Amount_scaled']
X['Time'] = data['Time_scaled']

# Nhãn dữ liệu
y = data['Class']

In [8]:
# Chọn 10,000 dòng dữ liệu bình thường cho tập huấn luyện
train_data_normal = data[data['Class'] == 0].sample(n=10000, random_state=42)

# Chọn 200 dòng dữ liệu bình thường và 200 dòng dữ liệu bất thường cho tập kiểm tra
test_data_normal = data[data['Class'] == 0].sample(n=200, random_state=42)
test_data_fraud = data[data['Class'] == 1].sample(n=200, random_state=42)

# Kết hợp dữ liệu test (200 dòng bình thường + 200 dòng bất thường)
test_data = pd.concat([test_data_normal, test_data_fraud])

# Tạo các tập X_train và X_test
X_train = train_data_normal.drop(columns=['Class', 'Amount', 'Time'])
X_train['Amount'] = train_data_normal['Amount_scaled']
X_train['Time'] = train_data_normal['Time_scaled']

X_test = test_data.drop(columns=['Class', 'Amount', 'Time'])
X_test['Amount'] = test_data['Amount_scaled']
X_test['Time'] = test_data['Time_scaled']

# Nhãn dữ liệu kiểm tra (class 0 và 1)
y_test = test_data['Class']

In [9]:
# Chuyển đổi dữ liệu huấn luyện và kiểm tra sang tensor cho PyTorch
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

In [10]:
import torch.nn as nn
import torch.optim as optim

class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2_mu = nn.Linear(32, latent_dim)
        self.fc2_logvar = nn.Linear(32, latent_dim)
        self.fc3 = nn.Linear(latent_dim, 32)
        self.fc4 = nn.Linear(32, input_dim)

    def encode(self, x):
        h = torch.relu(self.fc1(x))
        return self.fc2_mu(h), self.fc2_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Khởi tạo mô hình VAE
input_dim = X_train.shape[1]
latent_dim = 2
model = VAE(input_dim, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Huấn luyện mô hình
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

epochs = 20
model.train()
train_loader = DataLoader(TensorDataset(X_train_tensor, X_train_tensor), batch_size=32, shuffle=True)

for epoch in range(epochs):
    train_loss = 0
    for x, _ in train_loader:
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(x)
        loss = loss_function(recon_batch, x, mu, logvar)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {train_loss / len(train_loader.dataset)}')

Epoch 1, Loss: 34.69888970336914
Epoch 2, Loss: 32.268689965820315
Epoch 3, Loss: 31.739519934082033
Epoch 4, Loss: 31.47432975769043
Epoch 5, Loss: 31.318169177246094
Epoch 6, Loss: 31.276053399658203
Epoch 7, Loss: 31.211805157470703
Epoch 8, Loss: 31.195224710083007
Epoch 9, Loss: 31.188827709960936
Epoch 10, Loss: 31.1304010559082
Epoch 11, Loss: 31.106282763671874
Epoch 12, Loss: 31.078238784790038
Epoch 13, Loss: 31.00732177429199
Epoch 14, Loss: 30.92915082397461
Epoch 15, Loss: 30.880446148681642
Epoch 16, Loss: 30.83421510620117
Epoch 17, Loss: 30.779971621704103
Epoch 18, Loss: 30.761000817871093
Epoch 19, Loss: 30.734686810302733
Epoch 20, Loss: 30.742814221191406


In [11]:
# Đặt mô hình vào chế độ đánh giá
model.eval()
with torch.no_grad():
    X_test_pred, _, _ = model(X_test_tensor)
    reconstruction_loss = torch.mean((X_test_tensor - X_test_pred) ** 2, dim=1).numpy()

# Thiết lập ngưỡng phát hiện bất thường
threshold = np.percentile(reconstruction_loss, 95)  # Ngưỡng 95%

# Dự đoán nhãn dựa trên ngưỡng
y_pred = (reconstruction_loss > threshold).astype(int)  # 1 là bất thường, 0 là bình thường

In [12]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# So sánh kết quả với nhãn thực tế
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))

Confusion Matrix:
 [[200   0]
 [180  20]]
Classification Report:
               precision    recall  f1-score   support

           0       0.53      1.00      0.69       200
           1       1.00      0.10      0.18       200

    accuracy                           0.55       400
   macro avg       0.76      0.55      0.44       400
weighted avg       0.76      0.55      0.44       400

ROC AUC Score: 0.55
